In [94]:
import pandas as pd
import numpy as np
import nltk
from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.linear_model import RidgeClassifier, LogisticRegression, RidgeClassifierCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.dummy import DummyClassifier
from nltk import word_tokenize, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from tqdm import tqdm, tqdm_pandas
import string
import re
import gensim
import collections

%matplotlib inline

In [95]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
sample_submission_data = pd.read_csv('data/sample_submission.csv')

In [96]:
train_data.head()
train_data.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

### Short exploration

In [97]:
print(train_data.comment_text[5000])
print(train_data.iloc[5000][['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']],'\n')

print(train_data.comment_text[4000])
print(train_data.iloc[4000][['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']],'\n')

print(train_data.comment_text[3000])
print(train_data.iloc[3000][['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']],'\n')

hay bitch 

thank you kindly for your advice on my vandalism but if your the dick who removed the thing abouth Berties make up costs Thats true... so ah FUCK  YOU
toxic            1
severe_toxic     0
obscene          1
threat           0
insult           1
identity_hate    0
Name: 5000, dtype: object 

"

Marmot, you're so cool! keep up the good work.
- ""a/s/l?"""
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 4000, dtype: object 

The Billy the Kid article with my contributions has been vandalized on or about Dec. 14, 2015 by someone calling themselves KrakatoaKatie. What had been the most detailed, and accurate account of Billy the Kid's life has been replaced with pure garbage. I ask every to look at the last version I submitted one concerned and compare it to what it has been replaced with. I ask anyone who is knowledgeable about the subject, and who doesn't have some sort of agenda, to take a look at what h

### Clean data: tokenize, remove stop words

In [98]:
stopset = set(stopwords.words('english'))
snow = SnowballStemmer('english')
WNlemma = WordNetLemmatizer()

def clean_text(x, normalization='stemming', remove_stop=False):
    """Function to preprocess text data. Removes punctuation and numbers. 
    Lemmatizes or stems words, depending on given parameter. Can also remove 
    stopwords if specified.
    
    Args:
        x (str): The piece of text to process.
        normalization (str): how to normalize words, 'stemming' (default) or 'lemmatization'.
        remove_stop (bool): whether to remove stopwords. Default is False.
        
    Returns:
        str: Preprocessed tokens, re-joined with spaces.
    """
    # split text
    words = word_tokenize(x)
    
    # remove punctuation and numbers
    words = [word for word in words if word not in string.punctuation and not bool(re.search(r'\d', word))]
    
    if normalization == 'stemming':
        words = [snow.stem(t) for t in words] # stemming
    elif normalization == 'lemmatization':
        words = [WNlemma.lemmatize(t.lower()) for t in words] # lemmatize words (advanced stemming)
    else:
        return 'Invalid parameter for normalization'
    
    # remove stop words
    if remove_stop:
        words = [word for word in words if word not in stopset]
    
    joined_words = ' '.join(words).replace('_', '')
    
    return joined_words

Apply text cleaning to column

In [99]:
tqdm.pandas(tqdm()) # for tracking progress
train_data['comment_text'] = train_data['comment_text'].progress_apply(lambda x: clean_text(x, normalization='lemmatization'))


0it [00:00, ?it/s]

  0%|                                                                                       | 0/159571 [00:00<?, ?it/s]

  0%|                                                                             | 59/159571 [00:00<04:34, 580.41it/s]

  0%|                                                                            | 121/159571 [00:00<04:29, 590.86it/s]

  0%|                                                                            | 169/159571 [00:00<04:51, 546.86it/s]

  0%|                                                                            | 242/159571 [00:00<04:30, 588.48it/s]

  0%|▏                                                                           | 337/159571 [00:00<04:00, 662.92it/s]

  0%|▏                                                                           | 421/159571 [00:00<03:44, 707.64it/s]

  0%|▏                                                                           | 489/159571 [00:00<03:51, 688.43it/s]

  0%|▎     

  3%|██▌                                                                        | 5368/159571 [00:07<04:10, 616.64it/s]

  3%|██▌                                                                        | 5441/159571 [00:07<03:58, 646.73it/s]

  3%|██▌                                                                        | 5508/159571 [00:07<04:15, 602.39it/s]Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\users\klein535\appdata\local\programs\python\python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\users\klein535\appdata\local\programs\python\python36\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "c:\users\klein535\appdata\local\programs\python\python36\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration


  6%|████▋                                                                     | 10092/159571 [0

 10%|███████▏                                                                  | 15547/159571 [00:22<03:00, 799.18it/s]


 13%|█████████▋                                                                | 20992/159571 [00:30<02:39, 869.19it/s]


 17%|████████████▍                                                             | 26934/159571 [00:37<03:11, 691.01it/s]


...

 89%|█████████████████████████████████████████████████████████████████        | 142160/159571 [03:15<00:29, 594.54it/s]


 92%|███████████████████████████████████████████████████████████████████      | 146558/159571 [03:23<00:24, 539.76it/s]


 95%|█████████████████████████████████████████████████████████████████████    | 151010/159571 [03:30<00:14, 604.63it/s]


 98%|███████████████████████████████████████████████████████████████████████▏ | 155704/159571 [03:38<00:06, 620.06it/s]


100%|█████████████████████████████████████████████████████████████████████████| 159571/159571 [03:44<00:00, 710.66it/s]

In [100]:
clean_text('This is a __test FUCK 99 !! .. fUcking ObSCENE languages shitz0r.', normalization='lemmatization')
# note: words with numbers in them currently get dropped. 
# suggestion: replace numbers in words with letters (e.g. 0 = o, 1 = i, 7 = t, 3 = e)

'this is a test fuck .. fucking obscene language'

### Explore N-gram frequencies to better estimate appropriate min_df parameter for model

In [101]:
vect = CountVectorizer(ngram_range=(1,2))
train_vect = vect.fit_transform(train_data['comment_text'])
dist = np.sum(train_vect, axis=0).tolist()[0]
vocab = vect.get_feature_names()

In [102]:
ngram_freq = {}

for tag, count in zip(vocab, dist):
    ngram_freq[tag]=count
    
counts = collections.Counter(list(ngram_freq.values()))

In [103]:
# freq, occurrences of freq
# e.g. 2045516 words occur one time
counts.most_common()[:10]

[(1, 1512510),
 (2, 277999),
 (3, 111360),
 (4, 61127),
 (5, 39095),
 (6, 27197),
 (7, 19998),
 (8, 15470),
 (9, 12206),
 (10, 9907)]

### Split data

In [104]:
X = train_data['comment_text']
y = train_data[['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']]

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Pipeline a few models

In [106]:
NB_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2), min_df=4, max_df=0.5, max_features=50000)),
                    ('tfidf', TfidfTransformer(use_idf=True)),
                    ('clf', OneVsRestClassifier(MultinomialNB(alpha=0.01), n_jobs=-1))])

SVC_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2), min_df=4, max_df=0.5, max_features=50000)),
                         ('tfidf', TfidfTransformer(use_idf=True)),
                       ('clf', OneVsRestClassifier(SVC(C=10, probability=True), n_jobs=-1))])

logistic_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2), min_df=4, max_df=0.5, max_features=15000)),
                         ('tfidf', TfidfTransformer(use_idf=True)),
                       ('clf', OneVsRestClassifier(LogisticRegression(C=0.1, class_weight='balanced'), n_jobs=-1))])

### Hyperparameter tuning

In [107]:
# Define parameters, specify for which part of pipeline with prefix, e.g. 'vect__'
SVC_params = {'vect__ngram_range': [(1,2)],
              'tfidf__use_idf': [True],
              'clf__estimator__C':[0.1, 1, 10]}

logistic_params = {#'vect__ngram_range': [(1,2)],
                   'vect__min_df': [3, 4, 5, 6],
                   'vect__max_df': [0.3, 0.4, 0.5, 0.6],
                   #'vect__max_features': [25000, 50000, 100000, None],
                    #'vect__max_features': [5000, 7500, 10000, 12500],
                  #'tfidf__use_idf': [True],
                  #'clf__estimator__C':[0.1, 0.3, 0.6, 1, 3],
                  #'clf__estimator__class_weight':['balanced', None],
                  #'clf__estimator__penalty':['l1', 'l2']
                  }

NB_params = {'vect__ngram_range': [(1,2)],
              'tfidf__use_idf': [True],
              'clf__estimator__alpha':[0.1, 1, 10]}

### New pipeline with optimal parameters

In [108]:
Pipeline([('vect', CountVectorizer(ngram_range=(1,2), min_df=6, max_df=0.3, max_features=25000)),
                         ('tfidf', TfidfTransformer(use_idf=True)),
                       ('clf', OneVsRestClassifier(LogisticRegression(C=0.1, class_weight='balanced'), n_jobs=-1))])

### Validation

In [109]:
# models to test
models = {'Logistic regression': logistic_clf,
          #'SVC': SVC_clf,
         #'Naïve Bayes': NB_clf
         }

In [110]:
# Benchmark score (in case of all 0 predictions):
pred = np.zeros(y_test.shape)
roc_auc_score(y_test, pred)

0.5

In [111]:
for model_name, model in models.items():
    print('Training {}...'.format(model_name))
    clf = model.fit(X_train, y_train)
    
    y_pred = clf.predict(X_train)
    print('{} train ROC_AUC score: {}'.format(model_name, roc_auc_score(y_train, y_pred)))
    
    y_pred = clf.predict(X_test)
    print('{} test ROC_AUC score: {}'.format(model_name, roc_auc_score(y_test, y_pred)))
    print('{} cross validation ROC_AUC score on 5 folds: {}'.format(model_name, cross_val_score(model, X, y, scoring='roc_auc', cv=5, n_jobs=-1).mean()))
    print('')

Training Logistic regression...
Logistic regression train ROC_AUC score: 0.9553583845455703
Logistic regression test ROC_AUC score: 0.9066418804422969
Logistic regression cross validation ROC_AUC score on 5 folds: 0.9765465005353489



#### Validation log

### Final parameter tuning (manual)

In [112]:
logistic_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2), min_df=6, max_df=0.3, max_features=25000)),
                         ('tfidf', TfidfTransformer(use_idf=True)),
                       ('clf', OneVsRestClassifier(LogisticRegression(C=0.1, class_weight='balanced'), n_jobs=-1))])

print('Cross_val_score with C=0.1, max_features=25000, max_df=0.3, min_df=6: ', cross_val_score(logistic_clf, X, y, scoring='roc_auc', cv=5, n_jobs=-1).mean())

logistic_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,2), min_df=5, max_df=0.3, max_features=25000)),
                         ('tfidf', TfidfTransformer(use_idf=True)),
                       ('clf', OneVsRestClassifier(LogisticRegression(C=0.1, class_weight='balanced'), n_jobs=-1))])

print('Cross_val_score with C=0.1, max_features=25000, max_df=0.4, min_df=5: ', cross_val_score(logistic_clf, X, y, scoring='roc_auc', cv=5, n_jobs=-1).mean())

#Cross_val_score with C=0.1:  0.9759436463859725
#Cross_val_score with C=0.01:  0.9635830622046198

#Cross_val_score with C=0.1, max_features=25000, max_df=0.5, min_df=4:  0.9760136616571229
#Cross_val_score with C=0.1, max_features=50000, max_df=0.5, min_df=4:  0.9759436463859725

#Cross_val_score with C=0.1, max_features=25000, max_df=0.3, min_df=4:  0.9762885351187846
#Cross_val_score with C=0.1, max_features=25000, max_df=0.4, min_df=4:  0.9760047347166096

#Cross_val_score with C=0.1, max_features=25000, max_df=0.3, min_df=6:  0.9763128416184881
#Cross_val_score with C=0.1, max_features=25000, max_df=0.3, min_df=5:  0.976302536076927

# Same test, but with correction in clean_text function (fixed lowercase issue):
# Cross_val_score with C=0.1, max_features=25000, max_df=0.3, min_df=6:  0.977167248668523
# Cross_val_score with C=0.1, max_features=25000, max_df=0.4, min_df=5:  0.9771515047562769

Cross_val_score with C=0.1, max_features=25000, max_df=0.3, min_df=6:  0.977167248668523
Cross_val_score with C=0.1, max_features=25000, max_df=0.4, min_df=5:  0.9771515047562769


To do: write automated test function (start with default, set params to best so far, test one param per iteration)

### Declare model with final parameters to use

In [113]:
final_model = Pipeline([('vect', CountVectorizer(ngram_range=(1,2), min_df=6, max_df=0.3, max_features=25000)),
                         ('tfidf', TfidfTransformer(use_idf=True)),
                       ('clf', OneVsRestClassifier(LogisticRegression(C=0.1, class_weight='balanced'), n_jobs=-1))])

### Make predictions for submission and save

Note: submission should be probabilities

In [114]:
print(test_data.comment_text[0])

Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,


In [115]:
# clean test data
test_data['comment_text'] = test_data['comment_text'].progress_apply(lambda x: clean_text(x, normalization='lemmatization'))


  3%|██▏                                                                        | 4455/153164 [00:07<03:26, 718.91it/s]


  6%|████▌                                                                      | 9245/153164 [00:14<03:40, 652.81it/s]


  9%|███████                                                                   | 14535/153164 [00:22<03:08, 734.95it/s]


 13%|█████████▌                                                                | 19718/153164 [00:30<03:27, 643.49it/s]


 17%|████████████▎                                                             | 25362/153164 [00:37<02:52, 741.47it/s]


...

 91%|██████████████████████████████████████████████████████████████████       | 138642/153164 [03:22<00:20, 696.91it/s]


 94%|████████████████████████████████████████████████████████████████████▌    | 143736/153164 [03:30<00:15, 592.06it/s]


 97%|██████████████████████████████████████████████████████████████████████▍  | 147872/153164 [03:37<00:07, 675.71it/s]


100%|████████████████████████████████████████████████████████████████████████▉| 152952/153164 [03:45<00:00, 680.05it/s]


100%|█████████████████████████████████████████████████████████████████████████| 153164/153164 [03:45<00:00, 678.53it/s]

In [116]:
print(test_data.comment_text[0])

yo bitch ja rule is more succesful then you 'll ever be whats up with you and hating you sad mofuckas ... i should bitch slap ur pethedic white face and get you to kiss my as you guy sicken me ja rule is about pride in da music man dont dis that shit on him and nothin is wrong bein like tupac he wa a brother too ... fuckin white boy get thing right next time.


In [117]:
sample_submission_data.head(2)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5


In [118]:
y_pred_final = final_model.fit(X, y).predict_proba(test_data['comment_text'])

In [119]:
predictions = pd.DataFrame(y_pred_final, columns=y_test.columns)

In [120]:
predictions.head(2)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.995390,0.942465,0.996497,0.866189,0.987634,0.947539
1,0.116185,0.063854,0.073821,0.039130,0.103398,0.085246


In [121]:
submission = pd.concat([test_data['id'], predictions], axis=1)

In [122]:
print('ID 00001cee341fdb12:\n',test_data.comment_text[0], '\n')
print('ID 0000247867823ef7:\n',test_data.comment_text[1])
submission.head(2)

ID 00001cee341fdb12:
 yo bitch ja rule is more succesful then you 'll ever be whats up with you and hating you sad mofuckas ... i should bitch slap ur pethedic white face and get you to kiss my as you guy sicken me ja rule is about pride in da music man dont dis that shit on him and nothin is wrong bein like tupac he wa a brother too ... fuckin white boy get thing right next time. 

ID 0000247867823ef7:
 == from rfc == the title is fine a it is imo


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.995390,0.942465,0.996497,0.866189,0.987634,0.947539
1,0000247867823ef7,0.116185,0.063854,0.073821,0.039130,0.103398,0.085246


In [123]:
submission.shape

(153164, 7)

In [124]:
TARGET_PATH = './data/submission_simple.csv'
submission.to_csv(TARGET_PATH, index=False)

My highest public leaderboard score on Kaggle: 0.0.9723 (Good for 3081nd place)

No. 1 score on the leaderboard: 0.9889

### Bonus: most important words per label 

In [174]:
all_estimators = final_model.named_steps['clf'].estimators_
vocab = final_model.named_steps['vect'].vocabulary_
index_to_words = {value: key for key,value in vocab.items()}

In [218]:
for index, label in enumerate(y.columns):
    print('Current label: {}'.format(label))
    words = {}
    coefs = all_estimators[index].coef_[0]
    for key in index_to_words.keys():
        words[index_to_words[key]] = coefs[key]
    words = sorted(words.items(), key=lambda x:x[1], reverse=True)
    top_5_most_important = words[:10]
    top_5_least_important = words[-10:]
    print('Top 10 most {} words:'.format(label))
    for pair in top_5_most_important:
        print(pair)
    print('')
    print('Top 10 least {} words:'.format(label))
    for pair in top_5_least_important:
        print(pair)
    print('\n')

Current label: toxic
Top 10 most toxic words:
('fuck', 9.295420547748519)
('fucking', 8.01926826032406)
('stupid', 7.775941645246558)
('idiot', 7.415786121096115)
('shit', 7.325892787270488)
('suck', 6.230368382977478)
('as', 5.351426184985407)
('asshole', 5.212978738540356)
('crap', 5.074895273248084)
('dick', 5.033463892230288)

Top 10 least toxic words:
('but', -2.018151832647545)
('at', -2.0247040154355096)
('source', -2.086360135356195)
('thank you', -2.116636549977007)
('thank', -2.1968729718784465)
('utc', -2.29552975080801)
('article', -2.5343513950197027)
('please', -2.843646044913631)
('talk', -3.364017510262568)
('thanks', -3.3772175055523883)


Current label: severe_toxic
Top 10 most severe_toxic words:
('fucking', 9.139312853190223)
('fuck', 9.003776593117689)
('bitch', 5.724100840073539)
('asshole', 5.601583500585075)
('shit', 5.515879511526424)
('suck', 5.185368550320496)
('dick', 4.986553179438271)
('as', 4.767529613564076)
('fucker', 4.682142091935353)
('faggot', 4.530